In [1]:
import pandas as pd

### Data

In [2]:
course = pd.read_csv("course.csv") 
course_element = pd.read_csv("course_element.csv") 
course_module = pd.read_csv("course_module.csv") 
sol_log = pd.read_csv("solution_log.csv") 
u_course_progress = pd.read_csv("user_course_progress.csv") 
u_element_progress = pd.read_csv("user_element_progress.csv") 
u_module_progress = pd.read_csv("user_module_progress.csv")

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Preprocessing (убираем неактивных юзеров)

In [3]:
u_modules_achieved = u_course_progress.groupby('user_id')['modules_achieved'].sum().reset_index()
user_id = u_modules_achieved[u_modules_achieved['modules_achieved'] > 0]['user_id']

u_course_progress = u_course_progress.merge(user_id, on='user_id')
u_module_progress = u_module_progress.merge(user_id, on='user_id')
u_element_progress = u_element_progress.merge(user_id, on='user_id')

In [14]:
user_id

0                      1034
1                      3971
5        100119700110000274
6        100119700110000277
7        100119700110000283
                ...        
22885    100120190110548272
22886    100120190110550407
22892    100120190110590055
22893    100120190110592083
22895    100120190112111400
Name: user_id, Length: 11663, dtype: int64

---

### user_max_delta_sub_time

In [15]:
sub_time = u_element_progress.merge(sol_log, left_on='id', 
                     right_on='element_progress_id')[['user_id', 'submission_time', 'course_id']]

In [16]:
sub_time['submission_time'] = pd.to_datetime(sub_time['submission_time'])

In [17]:
sub_time.sort_values('submission_time', inplace=True)

In [19]:
sub_time['delta'] = sub_time.groupby(['user_id', 'course_id'])['submission_time'].diff()

In [20]:
sub_time['delta'] = sub_time['delta'].apply(lambda x: round(x.total_seconds(), 2))

In [22]:
u_max_delta_sub_time = sub_time.groupby(['user_id', 'course_id'])['delta'].max().reset_index()

In [25]:
u_max_delta_sub_time.to_csv('user_max_delta_sub_time.csv')

In [23]:
u_max_delta_sub_time

,user_id,course_id,delta
0,1034,3,1483061.97
1,3971,3,4503669.93
2,100119700110000274,3,5223606.22
3,100119700110000277,35,1112896.25
4,100119700110000277,237,5190404.96
...,...,...,...
13607,100120190110548272,3,112.89
13608,100120190110550407,3,2439.57
13609,100120190110590055,89,4936412.93
13610,100120190110592083,3,1935783.49


---